# Spark on Tour
## Introducción a la API Estructurada

Importamos librerías e inicializamos la sesión Spark.

In [1]:
import findspark
findspark.init()

import pyspark
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

from IPython.display import clear_output
import plotly.express as px

#setup spark session
sparkSession = (SparkSession.builder
                .appName("Introducción API estructurada")
                .master("local[*]")
                .config("spark.scheduler.mode", "FAIR")
                .getOrCreate())
sparkSession.sparkContext.setLogLevel("ERROR")

## Creación de Dataframes

In [2]:
movies = sparkSession.read.csv("/tmp/movielens/movies.csv", header=True)
movies.show(10)

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
+-------+--------------------+--------------------+
only showing top 10 rows



## Proyecciones: manipulando las columnas

### Seleccionando columnas

In [3]:
onlyTitle = movies.select("movieId", "title")
onlyTitle.show(10)

+-------+--------------------+
|movieId|               title|
+-------+--------------------+
|      1|    Toy Story (1995)|
|      2|      Jumanji (1995)|
|      3|Grumpier Old Men ...|
|      4|Waiting to Exhale...|
|      5|Father of the Bri...|
|      6|         Heat (1995)|
|      7|      Sabrina (1995)|
|      8| Tom and Huck (1995)|
|      9| Sudden Death (1995)|
|     10|    GoldenEye (1995)|
+-------+--------------------+
only showing top 10 rows



### Renombrando columnas

In [4]:
renamedColumns = movies.select(col("movieId").alias("id"), "title")
renamedColumns.show(10)

+---+--------------------+
| id|               title|
+---+--------------------+
|  1|    Toy Story (1995)|
|  2|      Jumanji (1995)|
|  3|Grumpier Old Men ...|
|  4|Waiting to Exhale...|
|  5|Father of the Bri...|
|  6|         Heat (1995)|
|  7|      Sabrina (1995)|
|  8| Tom and Huck (1995)|
|  9| Sudden Death (1995)|
| 10|    GoldenEye (1995)|
+---+--------------------+
only showing top 10 rows



In [5]:
renamedColumns = movies.withColumnRenamed("movieId", "id")
renamedColumns.show(10)

+---+--------------------+--------------------+
| id|               title|              genres|
+---+--------------------+--------------------+
|  1|    Toy Story (1995)|Adventure|Animati...|
|  2|      Jumanji (1995)|Adventure|Childre...|
|  3|Grumpier Old Men ...|      Comedy|Romance|
|  4|Waiting to Exhale...|Comedy|Drama|Romance|
|  5|Father of the Bri...|              Comedy|
|  6|         Heat (1995)|Action|Crime|Thri...|
|  7|      Sabrina (1995)|      Comedy|Romance|
|  8| Tom and Huck (1995)|  Adventure|Children|
|  9| Sudden Death (1995)|              Action|
| 10|    GoldenEye (1995)|Action|Adventure|...|
+---+--------------------+--------------------+
only showing top 10 rows



### Añadiendo/transformando columnas

Podemos añadir nuevas columnas a partir de las ya existentes utilizando funciones predefinidas de Spark

In [6]:
newColumns = movies.select("movieId", "title", "genres", col("title").startswith("Toy"))
newColumns.show(10)

+-------+--------------------+--------------------+----------------------+
|movieId|               title|              genres|startswith(title, Toy)|
+-------+--------------------+--------------------+----------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|                  true|
|      2|      Jumanji (1995)|Adventure|Childre...|                 false|
|      3|Grumpier Old Men ...|      Comedy|Romance|                 false|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|                 false|
|      5|Father of the Bri...|              Comedy|                 false|
|      6|         Heat (1995)|Action|Crime|Thri...|                 false|
|      7|      Sabrina (1995)|      Comedy|Romance|                 false|
|      8| Tom and Huck (1995)|  Adventure|Children|                 false|
|      9| Sudden Death (1995)|              Action|                 false|
|     10|    GoldenEye (1995)|Action|Adventure|...|                 false|
+-------+----------------

O podemos definir nuestras propias funciones de transformación

In [7]:
def extractYear(movieTitle):
    import re
    result = re.findall("\((.*?)\)", movieTitle)
    if (len(result) == 1) and (result[0].find('–') == -1):
        return int(result[0])
    else:
        return int(0)
extractYear_udf = udf(extractYear, IntegerType())

In [8]:
newColumns = movies.select("movieId", "title", "genres", extractYear_udf(col("title")).alias("year"))
newColumns.show(10)
newColumns.printSchema()

+-------+--------------------+--------------------+----+
|movieId|               title|              genres|year|
+-------+--------------------+--------------------+----+
|      1|    Toy Story (1995)|Adventure|Animati...|1995|
|      2|      Jumanji (1995)|Adventure|Childre...|1995|
|      3|Grumpier Old Men ...|      Comedy|Romance|1995|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|1995|
|      5|Father of the Bri...|              Comedy|1995|
|      6|         Heat (1995)|Action|Crime|Thri...|1995|
|      7|      Sabrina (1995)|      Comedy|Romance|1995|
|      8| Tom and Huck (1995)|  Adventure|Children|1995|
|      9| Sudden Death (1995)|              Action|1995|
|     10|    GoldenEye (1995)|Action|Adventure|...|1995|
+-------+--------------------+--------------------+----+
only showing top 10 rows

root
 |-- movieId: string (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)
 |-- year: integer (nullable = true)



## Filtrado: seleccionado filas

In [9]:
newColumns.filter("year > '2010'").show(10)

+-------+--------------------+--------------------+----+
|movieId|               title|              genres|year|
+-------+--------------------+--------------------+----+
|  82152|      Beastly (2011)|Drama|Fantasy|Rom...|2011|
|  83349|Green Hornet, The...|Action|Comedy|Cri...|2011|
|  83480|Season of the Wit...|Adventure|Drama|F...|2011|
|  83613|Cowboys & Aliens ...|Action|Sci-Fi|Thr...|2011|
|  83910| Dilemma, The (2011)|        Comedy|Drama|2011|
|  84152|    Limitless (2011)|     Sci-Fi|Thriller|2011|
|  84273|Zeitgeist: Moving...|         Documentary|2011|
|  84374|No Strings Attach...|      Comedy|Romance|2011|
|  84392|Lincoln Lawyer, T...|Crime|Drama|Thriller|2011|
|  84414|All-Star Superman...|Action|Adventure|...|2011|
+-------+--------------------+--------------------+----+
only showing top 10 rows



## Agregaciones

Nos permiten calcular resultados a partir de la agregación de múltiples filas

In [10]:
newColumns.select(countDistinct("year")).show()

+--------------------+
|count(DISTINCT year)|
+--------------------+
|                 105|
+--------------------+



In [11]:
from2010 = newColumns.filter("year > '2010'")
from2010.select(countDistinct("year")).show()

+--------------------+
|count(DISTINCT year)|
+--------------------+
|                   8|
+--------------------+



In [12]:
from2010.select(min("year"), max("year"), avg("year")).show()

+---------+---------+------------------+
|min(year)|max(year)|         avg(year)|
+---------+---------+------------------+
|     2011|     2018|2013.9655596555965|
+---------+---------+------------------+



## Agrupaciones y agregaciones
Nos permiten agrupar filas y, opcionalmente, calcular resultados agregados por grupos

In [13]:
newColumns\
    .select("year") \
    .groupBy("year") \
    .count(). \
    sort(desc("year")) \
    .show()

+----+-----+
|year|count|
+----+-----+
|2018|   41|
|2017|  147|
|2016|  218|
|2015|  274|
|2014|  271|
|2013|  219|
|2012|  216|
|2011|  240|
|2010|  216|
|2009|  248|
|2008|  243|
|2007|  259|
|2006|  264|
|2005|  246|
|2004|  240|
|2003|  238|
|2002|  268|
|2001|  234|
|2000|  249|
|1999|  235|
+----+-----+
only showing top 20 rows



## Joins
Combinar dierentes datasets en un nuevo dataset

In [14]:
links = sparkSession.read.csv("/tmp/movielens/links.csv", header=True)
links.show(10)

+-------+-------+------+
|movieId| imdbId|tmdbId|
+-------+-------+------+
|      1|0114709|   862|
|      2|0113497|  8844|
|      3|0113228| 15602|
|      4|0114885| 31357|
|      5|0113041| 11862|
|      6|0113277|   949|
|      7|0114319| 11860|
|      8|0112302| 45325|
|      9|0114576|  9091|
|     10|0113189|   710|
+-------+-------+------+
only showing top 10 rows



In [15]:
movies \
    .join(links, movies["movieId"] == links["movieId"]) \
    .drop(links["movieId"]) \
    .show(10)

+-------+--------------------+--------------------+-------+------+
|movieId|               title|              genres| imdbId|tmdbId|
+-------+--------------------+--------------------+-------+------+
|      1|    Toy Story (1995)|Adventure|Animati...|0114709|   862|
|      2|      Jumanji (1995)|Adventure|Childre...|0113497|  8844|
|      3|Grumpier Old Men ...|      Comedy|Romance|0113228| 15602|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|0114885| 31357|
|      5|Father of the Bri...|              Comedy|0113041| 11862|
|      6|         Heat (1995)|Action|Crime|Thri...|0113277|   949|
|      7|      Sabrina (1995)|      Comedy|Romance|0114319| 11860|
|      8| Tom and Huck (1995)|  Adventure|Children|0112302| 45325|
|      9| Sudden Death (1995)|              Action|0114576|  9091|
|     10|    GoldenEye (1995)|Action|Adventure|...|0113189|   710|
+-------+--------------------+--------------------+-------+------+
only showing top 10 rows

